In [1]:
import pandas as pd
import polars as pl
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mplfinance as mpf
from plotly.subplots import make_subplots
from dash import Dash, dcc, html
import pandas_market_calendars as mcal
nse = mcal.get_calendar('NSE')

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

polars.config.Config

In [2]:
import sys
sys.path.append('..')
from tooling.filter import find_atm, option_tool
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm
from tooling.enums import Index, AssetClass, StrikeSpread, Spot

In [3]:
def get_expiry(f_today):
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = '0'
    mth=expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime('%y%b').upper()
        return f'{symbol}{date_string}{strike}{opt_type}'
    else:
        if expiry.day<=9:
            date_string = f'{expiry.year - 2000}{mth}{temp}{expiry.day}'
        else :
            date_string = f'{expiry.year - 2000}{mth}{expiry.day}'
        return f'{symbol}{date_string}{strike}{opt_type}'

In [11]:
bnf = await fetch_spot_data(
    instrument='bnf',
    start_date=dt.date(2023, 1, 1),
    start_time=dt.time(9, 15),
    end_date=dt.date(2023, 1, 5),
    end_time=dt.time(15, 30),
)
bnf = bnf.to_pandas()

In [6]:
opt = await fetch_option_data(
    index='bnf',
    expiry=dt.date(2023, 7, 27),
    strike=45900,
    asset_class=AssetClass.CALL,
    start_date=dt.date(2023, 7, 27),
    start_time=dt.time(9, 15),
    end_date=dt.date(2023, 7, 27),
    end_time=dt.time(15, 30)
)
opt



index,expiry,strike,asset_class,datetime,o,h,l,c,v,oi
str,date,i64,str,datetime[μs],f64,f64,f64,f64,i64,i64
"""bnf""",2023-07-27,45900,"""C""",2023-07-27 09:15:00,275.5,358.35,275.5,335.45,389538,1257255
"""bnf""",2023-07-27,45900,"""C""",2023-07-27 09:16:00,338.35,349.65,323.5,331.5,284715,1257255
"""bnf""",2023-07-27,45900,"""C""",2023-07-27 09:17:00,336.0,339.5,320.9,333.45,171780,1217670
"""bnf""",2023-07-27,45900,"""C""",2023-07-27 09:18:00,333.35,350.0,333.35,346.8,207675,1217670
"""bnf""",2023-07-27,45900,"""C""",2023-07-27 09:19:00,346.5,370.7,344.5,359.7,311130,1217670
"""bnf""",2023-07-27,45900,"""C""",2023-07-27 09:20:00,362.75,371.15,358.8,361.0,224445,1151370
"""bnf""",2023-07-27,45900,"""C""",2023-07-27 09:21:00,360.7,368.45,353.5,364.45,211275,1151370
"""bnf""",2023-07-27,45900,"""C""",2023-07-27 09:22:00,364.9,377.0,364.1,372.0,225570,1151370
"""bnf""",2023-07-27,45900,"""C""",2023-07-27 09:23:00,371.1,372.5,364.25,364.25,140805,1145970


In [40]:
df = bnf

async def generate_trades():
    is_ce_trade_active = False
    is_pe_trade_active = False
    max_price = 0
    portfolio_value = 10_00_000
    trade_book = []
    ce_df = pd.DataFrame()
    pe_df = pd.DataFrame()
    atm_flag = False
    ce_base_price_flag = False
    pe_base_price_flag = False
    
    for i in range(1, len(df)):
        ce_points = 0
        pe_points = 0
        points = 0
        print(df.iloc[i]['datetime'])
        if df.iloc[i]['datetime'].time() >= dt.time(9, 30) and df.iloc[i]['datetime'].time() <= dt.time(15, 0) and (not is_ce_trade_active):
            signal_time = df.iloc[i]['datetime'].time()
            if not atm_flag:
                atm = int(df.iloc[i]['o'] / 100) * 100
                atm_flag = True
            current_date = df.iloc[i]['datetime'].date()
            expiry_date = get_expiry(current_date)
            
            ce_df = await fetch_option_data(
                index='bnf',
                expiry=expiry_date,
                strike=atm,
                asset_class=AssetClass.CALL,
                start_date=current_date,
                start_time=dt.time(9, 15),
                end_date=current_date,
                end_time=dt.time(15, 30)
            )
    
            pe_df = await fetch_option_data(
                index='bnf',
                expiry=expiry_date,
                strike=atm,
                asset_class=AssetClass.PUT,
                start_date=current_date,
                start_time=dt.time(9, 15),
                end_date=current_date,
                end_time=dt.time(15, 30)
            )

            ce_df = ce_df.to_pandas()
            pe_df = pe_df.to_pandas()

            # print(ce_df, pe_df)

            if not ce_base_price_flag:
                ce_base_price = ce_df.loc[ce_df['datetime'].dt.time == signal_time, 'c'].values[0]
                ce_base_price_flag = True
            if not pe_base_price_flag:
                pe_base_price = pe_df.loc[pe_df['datetime'].dt.time == signal_time, 'c'].values[0]
                pe_base_price_flag = True

            print(ce_base_price, pe_base_price)
        
        if not is_ce_trade_active:
            for j in range(1, len(ce_df)):
                if ce_df.iloc[j]['datetime'].time() > signal_time:
                    if ce_df.iloc[j]['o'] >= ce_base_price * 1.12 and not is_ce_trade_active:
                        # Entry Triggered
                        print(ce_df.iloc[j]['datetime'], 'Entry Triggered CE')
                        is_ce_trade_active = True
                        ce_entry_price = ce_df.iloc[j]['o']
                        ce_sl_price = ce_df.iloc[j]['o'] * 0.92
                        ce_initial_sl = ce_sl_price
                        ce_atm = atm
                        ce_entry_time = ce_df.iloc[j]['datetime']
                        
                    if is_ce_trade_active:
                        max_price = max(max_price, df.iloc[j]['o'])
                        if ce_df.iloc[j]['o'] > ce_df.iloc[j-1]['o'] and ce_df.iloc[j]['o'] > max_price:
                            # Trailing
                            ce_sl_price += (ce_df.iloc[j]['o'] - ce_df.iloc[j-1]['o'])
                        k
                        if ce_df.iloc[j]['o'] < ce_sl_price:
                            # Stoploss Hit
                            print(ce_df.iloc[j]['datetime'], 'SL Hit')
                            is_ce_trade_active = False
                            ce_exit_price = ce_sl_price
                            ce_points = ce_exit_price - ce_entry_price
                            ce_exit_time = ce_df.iloc[j]['datetime']
                            break

                        elif df.iloc[j]['datetime'].time() >= dt.time(15, 20):
                            # EOD SquareOff
                            print(ce_df.iloc[j]['datetime'], 'EOD')
                            is_ce_trade_active = False
                            ce_exit_price = df.iloc[j]['o']
                            ce_points = ce_exit_price - ce_entry_price
                            ce_exit_time = ce_df.iloc[j]['datetime']
                            break

        if ce_points != 0:
            qty = 10000 / ce_entry_price # For 1% RPT
            points = ce_points * 0.95 if ce_points > 0 else ce_points * 1.05
            trade = {
                'Signal Generated At': signal_time,
                'Strike': ce_atm,
                'Entry Time': ce_entry_time,
                'Entry Price': ce_entry_price,
                'Initial SL': ce_initial_sl,
                'SL': ce_sl_price,
                'Exit Time': ce_exit_time,
                'Exit Price': ce_exit_price,
                'Points Captured': ce_points,
                'After Costs': points,
                'Qty': qty,
                'ROI%': ((points) * qty / portfolio_value) * 100,
                'Trade Year': ce_entry_time.year
            }
            print(trade)
            trade_book.append(trade)
            atm_flag = False
            
        # if not is_pe_trade_active:
        #     for k in range(1, len(pe_df)):
        #         if pe_df.iloc[k]['datetime'].time() > signal_time:
        #             if pe_df.iloc[k]['o'] >= pe_base_price * 1.12 and not is_pe_trade_active:
        #                 # Entry Triggered
        #                 print(ce_df.iloc[k]['datetime'], 'Entry Triggered PE')
        #                 is_pe_trade_active = True
        #                 pe_entry_price = pe_df.iloc[k]['o']
        #                 pe_sl_price = pe_df.iloc[k]['o'] * 0.92
        #                 pe_initial_sl = pe_sl_price
        #                 pe_atm = atm
        #                 pe_entry_time = pe_df.iloc[k]['datetime']

        #             if is_pe_trade_active:
        #                 if pe_df.iloc[k]['o'] > pe_df.iloc[k-1]['o']:
        #                     # Trailing Up
        #                     pe_sl_price += (pe_df.iloc[k]['o'] - pe_df.iloc[k-1]['o'])
                            
        #                 if pe_df.iloc[k]['o'] < pe_sl_price:
        #                     # Stoploss Hit
        #                     print(pe_df.iloc[k]['datetime'], 'SL Hit')
        #                     is_pe_trade_active = False
        #                     pe_exit_price = pe_sl_price
        #                     pe_points = pe_exit_price - pe_entry_price
        #                     pe_exit_time = pe_df.iloc[k]['datetime']
        #                     break

        #                 elif df.iloc[k]['datetime'].time() >= dt.time(15, 20):
        #                     # EOD SquareOff
        #                     print(pe_df.iloc[k]['datetime'], 'EOD')
        #                     is_pe_trade_active = False
        #                     pe_exit_price = df.iloc[k]['o']
        #                     pe_points = pe_exit_price - pe_entry_price
        #                     pe_exit_time = pe_df.iloc[k]['datetime']
        #                     break

        # if pe_points != 0:
        #     qty = 10000 / pe_entry_price # For 1% RPT
        #     points = pe_points * 0.95 if pe_points > 0 else pe_points * 1.05
        #     trade = {
        #         'Signal Generated At': signal_time,
        #         'Strike': pe_atm,
        #         'Entry Time': pe_entry_time,
        #         'Entry Price': pe_entry_price,
        #         'Initial SL': pe_initial_sl,
        #         'SL': pe_sl_price,
        #         'Exit Time': pe_exit_time,
        #         'Exit Price': pe_exit_price,
        #         'Points Captured': pe_points,
        #         'After Costs': points,
        #         'Qty': qty,
        #         'ROI%': ((points) * qty / portfolio_value) * 100,
        #         'Trade Year': pe_entry_time.year
        #     }
        #     print(trade)
        #     trade_book.append(trade)
        #     atm_flag = False

    return pd.DataFrame(trade_book)

tb = await generate_trades()
tb

2023-01-03 09:15:00
2023-01-03 09:16:00
2023-01-03 09:17:00
2023-01-03 09:18:00
2023-01-03 09:19:00
2023-01-03 09:20:00
2023-01-03 09:21:00
2023-01-03 09:22:00
2023-01-03 09:23:00
2023-01-03 09:24:00
2023-01-03 09:25:00
2023-01-03 09:26:00
2023-01-03 09:27:00
2023-01-03 09:28:00
2023-01-03 09:29:00
2023-01-03 09:30:00
327.25 203.15
2023-01-03 10:10:00 Entry Triggered CE
2023-01-03 11:39:00 SL Hit
{'Signal Generated At': datetime.time(9, 30), 'Strike': 43200, 'Entry Time': Timestamp('2023-01-03 10:10:00'), 'Entry Price': 374.55, 'Initial SL': 344.586, 'SL': 344.586, 'Exit Time': Timestamp('2023-01-03 11:39:00'), 'Exit Price': 344.586, 'Points Captured': -29.964, 'After Costs': -31.4622, 'Qty': 26.698705112802028, 'ROI%': -0.08399999999999999, 'Trade Year': 2023}
2023-01-03 12:34:00 Entry Triggered PE
2023-01-03 12:35:00 SL Hit
{'Signal Generated At': datetime.time(9, 30), 'Strike': 43200, 'Entry Time': Timestamp('2023-01-03 12:34:00'), 'Entry Price': 245.4, 'Initial SL': 225.76800000000

CancelledError: 